# MTCNN 

In [8]:


import cv2 as cv
import numpy as np
from mtcnn.mtcnn import MTCNN
detector = MTCNN()
cap = cv.VideoCapture(r'stairs.mp4')
# cap = cv.VideoCapture(r'capture_data.mp4')

while True:
    res,frame = cap.read()
    if not res:
        break
    faces = detector.detect_faces(frame)
    print(faces)
    if len(faces)>0:
        face = faces[0]
        fx,fy,fw,fh = face['box']
        cv.rectangle(frame, (fx,fy),(fx+fw,fy+fh),(0, 255, 0), 2)
    cv.imshow("capture",frame)
    if cv.waitKey(1) & 0xFF ==ord('q'):
        break
cap.release()
cv.destroyAllWindows()

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[{'box': [53, 199, 55, 72], 'confidence': 0.8326113224029541, 'keypoints': {'nose': [102, 236], 'mouth_right': [103, 249], 'right_eye': [100, 221], 'left_eye': [82, 225], 'mouth_left': [87, 252]}}]
[]
[]
[{'box': [64, 198, 58, 77], 'confidence': 0.914110541343689, 'keypoints': {'nose': [117, 236], 'mouth_right': [114, 254], 'right_eye': [116, 222], 'left_eye': [96, 222], 'mouth_left': [98, 254]}}]
[]
[]
[]
[]
[]


#  MTCNN Upgraded

In [2]:


import cv2 as cv
from facenet_pytorch import MTCNN
import torch
import threading
import time

# Use GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
detector = MTCNN(keep_all=True, device=device)

cap = cv.VideoCapture('stairs.mp4')

# Shared data
boxes = []
frame_resized_rgb = None
frame_original = None
lock = threading.Lock()
stop_thread = False

def detect_faces_thread():
    global boxes, frame_resized_rgb, stop_thread
    while not stop_thread:
        time.sleep(0.01)
        with lock:
            if frame_resized_rgb is not None:
                bxs, probs = detector.detect(frame_resized_rgb)
                boxes = bxs if bxs is not None else []

# Start detection thread
thread = threading.Thread(target=detect_faces_thread)
thread.start()

frame_count = 0
while True:
    res, frame = cap.read()
    if not res:
        break

    frame_original = frame.copy()
    frame_count += 1

    # Resize and convert to RGB
    small_frame = cv.resize(frame, (0, 0), fx=0.5, fy=0.5)
    small_rgb = cv.cvtColor(small_frame, cv.COLOR_BGR2RGB)

    # Send resized RGB frame every 5th frame
    if frame_count % 5 == 0:
        with lock:
            frame_resized_rgb = small_rgb.copy()

    # Draw boxes
    with lock:
        for box in boxes:
            x1, y1, x2, y2 = box
            # Scale coordinates back to original size
            x1, y1, x2, y2 = [int(coord * 2) for coord in [x1, y1, x2, y2]]
            cv.rectangle(frame_original, (x1, y1), (x2, y2), (0, 255, 0), 2)

    cv.imshow("capture", frame_original)

    if cv.waitKey(30) & 0xFF == ord('q'):
        break

stop_thread = True
thread.join()
cap.release()
cv.destroyAllWindows()


#  MTCNN upgraded ulta pro max

In [1]:
import cv2 as cv
from facenet_pytorch import MTCNN
import torch
import threading
import time
import queue

# Use GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
detector = MTCNN(keep_all=True, device=device)

cap = cv.VideoCapture('stairs.mp4')

# Shared Queue
# capture_queue = queue.Queue(maxsixe =1)
frame_queue = queue.Queue(maxsize=1)
result_queue = queue.Queue(maxsize=1)

stop_thread = False

def detect_faces():
    
    while not stop_thread:
        try:
            frame_rgb = frame_queue.get(timeout=1)
        except queue.Empty:
            continue
        boxes, _ = detector.detect(frame_rgb)
        result_queue.queue.clear()  # Keep only latest detection result
        result_queue.put(boxes if boxes is not None else [])

# Start detection thread
thread = threading.Thread(target=detect_faces)
thread.daemon = True
thread.start()

frame_count = 0
boxes = []
cap_faces = []

while True:
    res, frame = cap.read()
    if not res:
        break

    frame_original = frame.copy()
    frame_count += 1

    # Resize frame for detection
    small_frame = cv.resize(frame, (320, 240))  # Fixed smaller size
    small_rgb = cv.cvtColor(small_frame, cv.COLOR_BGR2RGB)

    # Every 5th frame, send to detector
#     if frame_count % 5 == 0 and not frame_queue.full():
    frame_queue.put(small_rgb)

    # Get detection results if available
    try:
        boxes = result_queue.get_nowait()
    except queue.Empty:
        pass

    # Draw boxes on original frame
    for box in boxes:
        x1, y1, x2, y2 = box
        # Scale coordinates from 320x240 to original size
        scale_x = frame.shape[1] / 320
        scale_y = frame.shape[0] / 240
        x1, y1, x2, y2 = [int(x1 * scale_x), int(y1 * scale_y), int(x2 * scale_x), int(y2 * scale_y)]
        face_region = frame_original[y1:y2, x1:x2]

        face_region = cv.resize(face_region, (160, 160))
        cap_faces.append(face_region)
        cv.rectangle(frame_original, (x1, y1), (x2, y2), (0, 255, 0), 2)

    cv.imshow("capture", frame_original)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

stop_thread = True
thread.join()
cap.release()
cv.destroyAllWindows()

In [3]:
# Small size
import cv2 as cv
from facenet_pytorch import MTCNN
import torch
import threading
import queue

# Use GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
detector = MTCNN(keep_all=True, device=device)

cap = cv.VideoCapture('stairs.mp4')

# Shared Queues
frame_queue = queue.Queue(maxsize=1)
result_queue = queue.Queue(maxsize=1)

stop_thread = False

def detect_faces():
    while not stop_thread:
        try:
            frame_rgb = frame_queue.get(timeout=1)
        except queue.Empty:
            continue
        boxes, _ = detector.detect(frame_rgb)
        result_queue.queue.clear()  # Keep only latest detection result
        result_queue.put(boxes if boxes is not None else [])

# Start detection thread
thread = threading.Thread(target=detect_faces)
thread.daemon = True
thread.start()

frame_count = 0
boxes = []
cap_faces = []

while True:
    res, frame = cap.read()
    if not res:
        break

    # Resize and convert to RGB immediately
    small_frame = cv.resize(frame, (500, 420))
    small_rgb = cv.cvtColor(small_frame, cv.COLOR_BGR2RGB)

    # Put frame in detection queue
    frame_queue.put(small_rgb)

    # Get detection results
    try:
        boxes = result_queue.get_nowait()
    except queue.Empty:
        pass

    # Draw detection boxes directly on small frame
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        face_region = small_frame[y1:y2, x1:x2]
        face_region = cv.resize(face_region, (160, 160))
        cap_faces.append(face_region)
        cv.rectangle(small_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Show processed small frame
    cv.imshow("capture", small_frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

stop_thread = True
thread.join()
cap.release()
cv.destroyAllWindows()


In [9]:
import cv2 as cv
from facenet_pytorch import MTCNN
import torch
import threading
import time
import queue

    # Use GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
detector = MTCNN(keep_all=True, device=device)

cap = cv.VideoCapture('stairs.mp4')

# Shared Queue
# capture_queue = queue.Queue(maxsixe =1)
frame_queue = queue.Queue(maxsize=1)
result_queue = queue.Queue(maxsize=1)

stop_thread = False

def detect_faces():
    
    while not stop_thread:
        try:
            frame_rgb = frame_queue.get(timeout=1)
        except queue.Empty:
            continue
        boxes, _ = detector.detect(frame_rgb)
        result_queue.queue.clear()  # Keep only latest detection result
        result_queue.put(boxes if boxes is not None else [])

# Start detection thread
thread = threading.Thread(target=detect_faces)
thread.daemon = True
thread.start()

frame_count = 0
boxes = []
cap_faces = []

while True:
    res, frame = cap.read()
    if not res:
        break

#     frame_original = frame.copy()
    frame_count += 1

    # Resize frame for detection
#     small_frame = cv.resize(frame, (320, 240))  # Fixed smaller size
#     small_rgb = cv.cvtColor(small_frame, cv.COLOR_BGR2RGB)
# 
    # Every 5th frame, send to detector
#     if frame_count % 5 == 0 and not frame_queue.full():
    frame_queue.put(frame)

    # Get detection results if available
    try:
        boxes = result_queue.get_nowait()
    except queue.Empty:
        pass

    # Draw boxes on original frame
    for box in boxes:
        x1, y1, x2, y2 = box
        # Scale coordinates from 320x240 to original size
#         scale_x = frame.shape[1] / 320
#         scale_y = frame.shape[0] / 240
#         x1, y1, x2, y2 = [int(x1 * scale_x), int(y1 * scale_y), int(x2 * scale_x), int(y2 * scale_y)]
        face_region = frame[y1:y2, x1:x2]

        face_region = cv.resize(face_region, (160, 160))
        cap_faces.append(frame)
        cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    cv.imshow("capture", frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

stop_thread = True
thread.join()
cap.release()
cv.destroyAllWindows()

In [2]:
import cv2 as cv
from facenet_pytorch import MTCNN
import torch
import threading
import time
import queue

# Use GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
detector = MTCNN(keep_all=True, device=device)

cap = cv.VideoCapture('stairs.mp4')

# Shared Queue
# capture_queue = queue.Queue(maxsixe =1)
frame_queue = queue.Queue(maxsize=1)
result_queue = queue.Queue(maxsize=1)

stop_thread = False

def detect_faces():
    
    while not stop_thread:
        try:
            frame_rgb = frame_queue.get(timeout=1)
        except queue.Empty:
            continue
        boxes, _ = detector.detect(frame_rgb)
        result_queue.queue.clear()  # Keep only latest detection result
        result_queue.put(boxes if boxes is not None else [])

# Start detection thread
thread = threading.Thread(target=detect_faces)
thread.daemon = True
thread.start()

frame_count = 0
boxes = []
cap_faces = []

while True:
    res, frame = cap.read()
    if not res:
        break

    frame_original = frame.copy()
    frame_count += 1

    # Resize frame for detection
    small_frame = cv.resize(frame, (320, 240))  # Fixed smaller size
    small_rgb = cv.cvtColor(small_frame, cv.COLOR_BGR2RGB)

    # Every 5th frame, send to detector
#     if frame_count % 5 == 0 and not frame_queue.full():
    frame_queue.put(small_rgb)

    # Get detection results if available
    try:
        boxes = result_queue.get_nowait()
    except queue.Empty:
        pass

    # Draw boxes on original frame
    for box in boxes:
        x1, y1, x2, y2 = box
        # Scale coordinates from 320x240 to original size
        scale_x = frame.shape[1] / 320
        scale_y = frame.shape[0] / 240
        x1, y1, x2, y2 = [int(x1 * scale_x), int(y1 * scale_y), int(x2 * scale_x), int(y2 * scale_y)]
        face_region = frame_original[y1:y2, x1:x2]

        face_region = cv.resize(face_region, (160, 160))
        cap_faces.append(face_region)
        cv.rectangle(frame_original, (x1, y1), (x2, y2), (0, 255, 0), 2)

    cv.imshow("capture", frame_original)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

stop_thread = True
thread.join()
cap.release()
cv.destroyAllWindows()

In [11]:
import cv2 as cv

# Assuming 'cropped_faces' is a list of images (NumPy arrays)
# For testing purposes, you can replace it with your list of cropped faces.
# cropped_faces = []  # Replace this with your list of cropped face images

# Check if the list is empty
if not cap_faces:
    print("No cropped faces available to view.")
else:
    index = 0  # Start from the first image in the list
    while True:
        # Display the current cropped face
        cv.imshow(f'Cropped Face {index+1}', cap_faces[index])
        print(f"Viewing Cropped Face {index+1}")

        # Wait for a key press
        key = cv.waitKey(0) & 0xFF

        if key == ord('n'):  # Press 'n' to view the next image
            index += 1
            if index >= len(cap_faces):  # If we reach the end of the list
                print("No more faces to view.")
                break
        elif key == ord('p'):  # Press 'p' to view the previous image
            index -= 1
            if index < 0:  # Prevent going out of bounds
                print("This is the first face.")
                index = 0
                
        elif key == ord('q'):  # Press 'q' to quit viewing
            print("Exiting view mode.")
            break

        # Close the window after key press
        cv.destroyAllWindows()

# Release any remaining OpenCV windows
cv.destroyAllWindows()


Viewing Cropped Face 1
Exiting view mode.


# MTCNN With SORT(face)

In [2]:
import cv2 as cv
from facenet_pytorch import MTCNN
import torch
import threading
import time
import queue
import numpy as np
# from sort import Sort  # Requires sort.py in your folder
import os
import sys
sys.path.append(r"D:\Nexus AI\Nexus Face recognition\testings\sort")
sort_path = os.path.join(os.getcwd(), 'sort')
sys.path.insert(0, sort_path)
from sort import Sort
import sys
# sys.path.insert(0, r"D:\Nexus AI\Nexus Face recognition\sort")
# from sort.sort import Sort



device = 'cuda' if torch.cuda.is_available() else 'cpu'
detector = MTCNN(keep_all=True, device=device)

cap = cv.VideoCapture('stairs.mp4')

frame_queue = queue.Queue(maxsize=1)
result_queue = queue.Queue(maxsize=1)
stop_thread = False

# Initialize SORT tracker
tracker = Sort()

def detect_faces():
    while not stop_thread:
        try:
            frame_rgb = frame_queue.get(timeout=1)
        except queue.Empty:
            continue
        boxes, _ = detector.detect(frame_rgb)
        # Format: [x1, y1, x2, y2]
        if boxes is None:
            boxes = []
        result_queue.queue.clear()
        result_queue.put(boxes)

thread = threading.Thread(target=detect_faces)
thread.daemon = True
thread.start()

frame_count = 0
boxes = []

while True:
    res, frame = cap.read()
    if not res:
        break

    frame_original = frame.copy()
    frame_count += 1

    h, w = frame.shape[:2]
    scale_w, scale_h = w / 320, h / 240

    # Resize for MTCNN
    small_frame = cv.resize(frame, (320, 240))
    small_rgb = cv.cvtColor(small_frame, cv.COLOR_BGR2RGB)

    # Send frame every 5th frame
    if frame_count % 5 == 0 and not frame_queue.full():
        frame_queue.put(small_rgb)

    # Get detection results
    try:
        raw_boxes = result_queue.get_nowait()
        # Scale up boxes to original resolution
        boxes = [[x1 * scale_w, y1 * scale_h, x2 * scale_w, y2 * scale_h] for x1, y1, x2, y2 in raw_boxes]
    except queue.Empty:
        pass

    # Update tracker with current boxes
    if len(boxes)>0:
        tracked_objects = tracker.update(np.array(boxes))  # shape: [n, 5] (x1, y1, x2, y2, ID)
    else:
        tracked_objects = []
        

    # Draw tracked boxes
    for x1, y1, x2, y2, track_id in tracked_objects:
        cv.rectangle(frame_original, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv.putText(frame_original, f'ID: {int(track_id)}', (int(x1), int(y1) - 10),
                   cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

    cv.imshow("capture", frame_original)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

stop_thread = True
thread.join()
cap.release()
cv.destroyAllWindows()


# faces in seperate file

In [22]:
import cv2 as cv
from facenet_pytorch import MTCNN
import torch
import threading
import queue
import numpy as np
import os
import sys

sys.path.append(r"D:\Nexus AI\Nexus Face recognition\testings\sort")
sort_path = os.path.join(os.getcwd(), 'sort')
sys.path.insert(0, sort_path)
from sort import Sort

device = 'cuda' if torch.cuda.is_available() else 'cpu'
detector = MTCNN(keep_all=True, device=device)

cap = cv.VideoCapture('stairs.mp4')

frame_queue = queue.Queue(maxsize=1)
result_queue = queue.Queue(maxsize=1)
stop_thread = False

# Initialize SORT tracker
tracker = Sort()

# List to store cropped faces
cropped_faces = []

def detect_faces():
    while not stop_thread:
        try:
            frame_rgb = frame_queue.get(timeout=1)
        except queue.Empty:
            continue
        boxes, _ = detector.detect(frame_rgb)
        # Format: [x1, y1, x2, y2]
        if boxes is None:
            boxes = []
        result_queue.queue.clear()
        result_queue.put(boxes)

thread = threading.Thread(target=detect_faces)
thread.daemon = True
thread.start()

frame_count = 0
boxes = []

while True:
    res, frame = cap.read()
    if not res:
        break

    frame_original = frame.copy()
    frame_count += 1

    h, w = frame.shape[:2]
    scale_w, scale_h = w / 320, h / 240

    # Resize for MTCNN
    small_frame = cv.resize(frame, (320, 240))
    small_rgb = cv.cvtColor(small_frame, cv.COLOR_BGR2RGB)

    # Send frame every 5th frame
    if frame_count % 5 == 0 and not frame_queue.full():
        frame_queue.put(small_rgb)

    # Get detection results
    try:
        raw_boxes = result_queue.get_nowait()
        # Scale up boxes to original resolution
        boxes = [[x1 * scale_w, y1 * scale_h, x2 * scale_w, y2 * scale_h] for x1, y1, x2, y2 in raw_boxes]
    except queue.Empty:
        pass

    # Update tracker with current boxes
    if len(boxes) > 0:
        tracked_objects = tracker.update(np.array(boxes))  # shape: [n, 5] (x1, y1, x2, y2, ID)
    else:
        tracked_objects = []

    # Crop faces and store them in the list
    for x1, y1, x2, y2, track_id in tracked_objects:
        # Draw the tracking box
        cv.rectangle(frame_original, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv.putText(frame_original, f'ID: {int(track_id)}', (int(x1), int(y1) - 10),
                   cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        
        # Crop the face from the frame
        face_crop = frame[int(y1):int(y2), int(x1):int(x2)]
        
        # Store the cropped face in the list
        cropped_faces.append(face_crop)

    cv.imshow("capture", frame_original)
    
    # Press 'q' to exit or 'v' to view cropped faces
    key = cv.waitKey(50) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('v'):  # View stored cropped faces
        for i, face in enumerate(cropped_faces):
            cv.imshow(f'Cropped Face {i+1}', face)
            cv.waitKey(0)  # Wait until a key is pressed to move to the next face
            cv.destroyAllWindows()

stop_thread = True
thread.join()
cap.release()
cv.destroyAllWindows()


In [12]:
cropped_faces

[array([[[ 15,  18,  18],
         [ 15,  18,  18],
         [ 16,  19,  19],
         ...,
         [138, 154, 147],
         [138, 154, 147],
         [137, 152, 148]],
 
        [[ 14,  17,  17],
         [ 14,  17,  17],
         [ 16,  19,  19],
         ...,
         [138, 154, 147],
         [138, 154, 147],
         [137, 152, 148]],
 
        [[ 13,  16,  16],
         [ 14,  17,  17],
         [ 15,  18,  18],
         ...,
         [140, 153, 147],
         [139, 152, 146],
         [137, 152, 148]],
 
        ...,
 
        [[ 22,  30,  39],
         [ 22,  28,  37],
         [ 20,  26,  35],
         ...,
         [122, 110, 117],
         [130, 118, 125],
         [122, 113, 117]],
 
        [[ 21,  28,  34],
         [ 21,  28,  34],
         [ 20,  27,  33],
         ...,
         [122, 111, 115],
         [131, 120, 124],
         [124, 116, 118]],
 
        [[ 18,  25,  31],
         [ 18,  25,  31],
         [ 18,  25,  31],
         ...,
         [118, 107, 111],
  

In [19]:
import cv2 as cv
from facenet_pytorch import MTCNN
import torch
import numpy as np
import os
import sys

sys.path.append(r"D:\Nexus AI\Nexus Face recognition\testings\sort")
sort_path = os.path.join(os.getcwd(), 'sort')
sys.path.insert(0, sort_path)
from sort import Sort

device = 'cuda' if torch.cuda.is_available() else 'cpu'
detector = MTCNN(keep_all=True, device=device)

cap = cv.VideoCapture('stairs.mp4')
tracker = Sort()
cropped_faces = []
frame_count = 0

while True:
    res, frame = cap.read()
    if not res:
        break

    frame_original = frame.copy()
    frame_count += 1

    h, w = frame.shape[:2]
    scale_w, scale_h = w / 320, h / 240

    # Resize for detection
    small_frame = cv.resize(frame, (320, 240))
    small_rgb = cv.cvtColor(small_frame, cv.COLOR_BGR2RGB)

    # Detect faces directly (synchronously)
    boxes, _ = detector.detect(small_rgb)
    if boxes is None:
        boxes = []
    boxes = [[x1 * scale_w, y1 * scale_h, x2 * scale_w, y2 * scale_h] for x1, y1, x2, y2 in boxes]

    # Track
    if len(boxes) > 0:
        tracked_objects = tracker.update(np.array(boxes))
    else:
        tracked_objects = []

    for x1, y1, x2, y2, track_id in tracked_objects:
        cv.rectangle(frame_original, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv.putText(frame_original, f'ID: {int(track_id)}', (int(x1), int(y1) - 10),
                   cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

        # Crop face from original frame
        face_crop = frame[int(y1):int(y2), int(x1):int(x2)]
        if face_crop.size > 0:
            cropped_faces.append(face_crop)

    cv.imshow("capture", frame_original)

    key = cv.waitKey(33) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('v'):
        for i, face in enumerate(cropped_faces):
            cv.imshow(f'Cropped Face {i + 1}', face)
            cv.waitKey(0)
            cv.destroyAllWindows()

cap.release()
cv.destroyAllWindows()


In [21]:
import cv2 as cv
from facenet_pytorch import MTCNN
import torch
import threading
import queue
import numpy as np
import os
import sys

sys.path.append(r"D:\Nexus AI\Nexus Face recognition\testings\sort")
sort_path = os.path.join(os.getcwd(), 'sort')
sys.path.insert(0, sort_path)
from sort import Sort

device = 'cuda' if torch.cuda.is_available() else 'cpu'
detector = MTCNN(keep_all=True, device=device)

cap = cv.VideoCapture('stairs.mp4')

frame_queue = queue.Queue(maxsize=1)
result_queue = queue.Queue(maxsize=1)
stop_thread = False
tracker = Sort()
cropped_faces = []

# Scale factors (will be updated dynamically)
scale_w, scale_h = 1.0, 1.0

def detect_faces():
    while not stop_thread:
        try:
            small_rgb, detection_frame, scale_w_, scale_h_ = frame_queue.get(timeout=1)
        except queue.Empty:
            continue

        boxes, _ = detector.detect(small_rgb)
        if boxes is None:
            boxes = []

        scaled_boxes = [[x1 * scale_w_, y1 * scale_h_, x2 * scale_w_, y2 * scale_h_] for x1, y1, x2, y2 in boxes]
        result_queue.queue.clear()
        result_queue.put((detection_frame, scaled_boxes))

thread = threading.Thread(target=detect_faces)
thread.daemon = True
thread.start()

frame_count = 0

while True:
    res, frame = cap.read()
    if not res:
        break

    frame_original = frame.copy()
    frame_count += 1

    h, w = frame.shape[:2]
    scale_w, scale_h = w / 320, h / 240

    small_frame = cv.resize(frame, (320, 240))
    small_rgb = cv.cvtColor(small_frame, cv.COLOR_BGR2RGB)

    # Send frame every 5th frame
    if frame_count % 5 == 0 and not frame_queue.full():
        frame_queue.put((small_rgb, frame_original.copy(), scale_w, scale_h))

    try:
        detection_frame, boxes = result_queue.get_nowait()
        if len(boxes) > 0:
            tracked_objects = tracker.update(np.array(boxes))
        else:
            tracked_objects = []

        for x1, y1, x2, y2, track_id in tracked_objects:
            cv.rectangle(frame_original, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv.putText(frame_original, f'ID: {int(track_id)}', (int(x1), int(y1) - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

            face_crop = detection_frame[int(y1):int(y2), int(x1):int(x2)]
            if face_crop.size > 0:
                cropped_faces.append(face_crop)

    except queue.Empty:
        pass

    cv.imshow("capture", frame_original)

    key = cv.waitKey(33) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('v'):
        for i, face in enumerate(cropped_faces):
            cv.imshow(f'Cropped Face {i + 1}', face)
            cv.waitKey(0)
            cv.destroyAllWindows()

stop_thread = True
thread.join()
cap.release()
cv.destroyAllWindows()


# view faces

In [23]:
import cv2 as cv

# Assuming 'cropped_faces' is a list of images (NumPy arrays)
# For testing purposes, you can replace it with your list of cropped faces.
# cropped_faces = []  # Replace this with your list of cropped face images

# Check if the list is empty
if not cropped_faces:
    print("No cropped faces available to view.")
else:
    index = 0  # Start from the first image in the list
    while True:
        # Display the current cropped face
        cv.imshow(f'Cropped Face {index+1}', cropped_faces[index])
        print(f"Viewing Cropped Face {index+1}")

        # Wait for a key press
        key = cv.waitKey(0) & 0xFF

        if key == ord('n'):  # Press 'n' to view the next image
            index += 1
            if index >= len(cropped_faces):  # If we reach the end of the list
                print("No more faces to view.")
                break
        elif key == ord('p'):  # Press 'p' to view the previous image
            index -= 1
            if index < 0:  # Prevent going out of bounds
                print("This is the first face.")
                index = 0
        elif key == ord('q'):  # Press 'q' to quit viewing
            print("Exiting view mode.")
            break

        # Close the window after key press
        cv.destroyAllWindows()

# Release any remaining OpenCV windows
cv.destroyAllWindows()


Viewing Cropped Face 1
Viewing Cropped Face 2
Viewing Cropped Face 3
Viewing Cropped Face 4
Viewing Cropped Face 5
Viewing Cropped Face 6
Viewing Cropped Face 7
Viewing Cropped Face 8
Viewing Cropped Face 9
Viewing Cropped Face 10
Viewing Cropped Face 11
Viewing Cropped Face 12
Viewing Cropped Face 13
Viewing Cropped Face 14
Viewing Cropped Face 15
Viewing Cropped Face 16
Viewing Cropped Face 17
Viewing Cropped Face 18
Viewing Cropped Face 19
Viewing Cropped Face 20
Viewing Cropped Face 21
Viewing Cropped Face 22
Viewing Cropped Face 23
Viewing Cropped Face 24
Viewing Cropped Face 25
Viewing Cropped Face 26
Viewing Cropped Face 27
Viewing Cropped Face 28
Viewing Cropped Face 29
Viewing Cropped Face 30
Viewing Cropped Face 31
Viewing Cropped Face 32
Viewing Cropped Face 33
Viewing Cropped Face 34
Viewing Cropped Face 35
Viewing Cropped Face 36
Viewing Cropped Face 37
Viewing Cropped Face 38
Viewing Cropped Face 39
Viewing Cropped Face 40
Viewing Cropped Face 41
Viewing Cropped Face 42
V

In [30]:
# Blaze face




In [13]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from typing import Tuple, Union
import math
import cv2
import numpy as np

MARGIN = 10  # pixels
ROW_SIZE = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
TEXT_COLOR = (255, 0, 0)

def _normalized_to_pixel_coordinates(
    normalized_x: float, normalized_y: float, image_width: int,
    image_height: int) -> Union[None, Tuple[int, int]]:

    def is_valid_normalized_value(value: float) -> bool:
        return (value > 0 or math.isclose(0, value)) and (value < 1 or math.isclose(1, value))

    if not (is_valid_normalized_value(normalized_x) and is_valid_normalized_value(normalized_y)):
        return None

    x_px = min(math.floor(normalized_x * image_width), image_width - 1)
    y_px = min(math.floor(normalized_y * image_height), image_height - 1)
    return x_px, y_px


def visualize(image, detection_result) -> np.ndarray:
    annotated_image = image.copy()
    height, width, _ = image.shape

    for detection in detection_result.detections:
        # Draw bounding box
        bbox = detection.bounding_box
        start_point = (bbox.origin_x, bbox.origin_y)
        end_point = (bbox.origin_x + bbox.width, bbox.origin_y + bbox.height)
        cv2.rectangle(annotated_image, start_point, end_point, TEXT_COLOR, 3)

        # Draw keypoints
        for keypoint in detection.keypoints:
            keypoint_px = _normalized_to_pixel_coordinates(keypoint.x, keypoint.y, width, height)
            if keypoint_px is not None:
                cv2.circle(annotated_image, keypoint_px, 2, (0, 255, 0), 2)

        # Draw label and score
        category = detection.categories[0]
        category_name = category.category_name if category.category_name else ''
        result_text = f"{category_name} ({category.score:.2f})"
        text_location = (MARGIN + bbox.origin_x, MARGIN + ROW_SIZE + bbox.origin_y)
        cv2.putText(annotated_image, result_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                    FONT_SIZE, TEXT_COLOR, FONT_THICKNESS)

    return annotated_image







# STEP 1: Import the necessary modules.
import numpy as np
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create a FaceDetector object.
base_options = python.BaseOptions(model_asset_path='detector.tflite')
options = vision.FaceDetectorOptions(base_options=base_options)
detector = vision.FaceDetector.create_from_options(options)

# STEP 3: Open the video file.
cap = cv2.VideoCapture('stairs.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert BGR to RGB (MediaPipe expects RGB)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # STEP 4: Create a MediaPipe Image from the NumPy frame.
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

    # STEP 5: Detect faces in the current frame.
    detection_result = detector.detect(mp_image)

    # STEP 6: Visualize the detection result.
    annotated_frame = visualize(frame, detection_result)  # visualize uses BGR, so pass original frame

    # Display the annotated frame.
    cv2.imshow('Detected Faces', annotated_frame)

    # Wait for 'q' key to exit, small delay for smoother video playback
    if cv2.waitKey(33) & 0xFF == ord('q'):
        break

# STEP 7: Release resources.
cap.release()
cv2.destroyAllWindows()


In [2]:
import cv2

IMAGE_FILE = 'image.jpg'
img = cv2.imread(IMAGE_FILE)

cv2.imshow("Image", img)  # Fix: provide window name as first argument
cv2.waitKey(0)            # Wait for key press
cv2.destroyAllWindows()   # Close the image window


In [43]:
# # STEP 1: Import the necessary modules.
# import numpy as np
# import cv2
# import mediapipe as mp
# from mediapipe.tasks import python
# from mediapipe.tasks.python import vision

# # STEP 2: Create a FaceDetector object.
# base_options = python.BaseOptions(model_asset_path='detector.tflite')
# options = vision.FaceDetectorOptions(base_options=base_options)
# detector = vision.FaceDetector.create_from_options(options)

# # STEP 3: Load the input image.
# cap = cv2.VideoCapture(r'stairs.mp4')
# while True:
#     res,frame = cap.read()
#     if not res :
#         break
#     IMAGE_FILE = frame
#     # IMAGE_FILE = 'image.jpg'
#     image = mp.Image.create_from_file(IMAGE_FILE)

#     # STEP 4: Detect faces in the input image.
#     detection_result = detector.detect(image)

#     # STEP 5: Visualize the detection result.
#     image_copy = np.copy(image.numpy_view())  # Convert to a mutable numpy image
#     annotated_image = visualize(image_copy, detection_result)

#     # Convert from BGR to RGB for OpenCV display (only if needed)
#     rgb_annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)

#     # Show image (local machine)
#     cv2.imshow("Detected Faces", rgb_annotated_image)
#     cv2.waitKey(0)
# cv2.destroyAllWindows()


TypeError: create_from_file(): incompatible function arguments. The following argument types are supported:
    1. (file_name: str) -> mediapipe.python._framework_bindings.image.Image

Invoked with: array([[[102,  94,  94],
        [184, 176, 176],
        [188, 185, 184],
        ...,
        [ 67,  63,  66],
        [ 69,  65,  68],
        [ 64,  60,  63]],

       [[ 39,  31,  31],
        [ 95,  87,  87],
        [166, 163, 162],
        ...,
        [ 67,  63,  66],
        [ 69,  65,  68],
        [ 64,  60,  63]],

       [[ 25,  17,  17],
        [ 45,  37,  37],
        [ 97,  94,  93],
        ...,
        [ 66,  62,  65],
        [ 69,  65,  68],
        [ 66,  62,  65]],

       ...,

       [[ 37,  40,  58],
        [ 37,  40,  58],
        [ 37,  40,  58],
        ...,
        [ 23,  28,  34],
        [ 23,  28,  34],
        [ 23,  28,  34]],

       [[ 37,  40,  58],
        [ 37,  40,  58],
        [ 37,  40,  58],
        ...,
        [ 23,  28,  34],
        [ 23,  

In [44]:
# STEP 1: Import the necessary modules.
import numpy as np
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create a FaceDetector object.
base_options = python.BaseOptions(model_asset_path='detector.tflite')
options = vision.FaceDetectorOptions(base_options=base_options)
detector = vision.FaceDetector.create_from_options(options)

# STEP 3: Open the video file.
cap = cv2.VideoCapture('stairs.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert BGR to RGB (MediaPipe expects RGB)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # STEP 4: Create a MediaPipe Image from the NumPy frame.
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

    # STEP 5: Detect faces in the current frame.
    detection_result = detector.detect(mp_image)

    # STEP 6: Visualize the detection result.
    annotated_frame = visualize(frame, detection_result)  # visualize uses BGR, so pass original frame

    # Display the annotated frame.
    cv2.imshow('Detected Faces', annotated_frame)

    # Wait for 'q' key to exit, small delay for smoother video playback
    if cv2.waitKey(33) & 0xFF == ord('q'):
        break

# STEP 7: Release resources.
cap.release()
cv2.destroyAllWindows()


In [30]:
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# Constants
MARGIN = 10  # pixels
ROW_SIZE = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
TEXT_COLOR = (255, 0, 0)

# Helper function to convert normalized coordinates to pixel coordinates
def _normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
    if not (0.0 <= normalized_x <= 1.0 and 0.0 <= normalized_y <= 1.0):
        return None
    x_px = min(int(normalized_x * image_width), image_width - 1)
    y_px = min(int(normalized_y * image_height), image_height - 1)
    return x_px, y_px

# Function to visualize detection results
def visualize(image, detection_result):
    annotated_image = image.copy()
    height, width, _ = image.shape

    for detection in detection_result.detections:
        bbox = detection.bounding_box
        start_point = (bbox.origin_x, bbox.origin_y)
        end_point = (bbox.origin_x + bbox.width, bbox.origin_y + bbox.height)
        cv2.rectangle(annotated_image, start_point, end_point, TEXT_COLOR, 2)

        for keypoint in detection.keypoints:
            keypoint_px = _normalized_to_pixel_coordinates(keypoint.x, keypoint.y, width, height)
            if keypoint_px:
                cv2.circle(annotated_image, keypoint_px, 2, (0, 255, 0), -1)

        category = detection.categories[0]
        category_name = category.category_name or ''
        probability = round(category.score, 2)
        result_text = f"{category_name} ({probability})"
        text_location = (bbox.origin_x + MARGIN, bbox.origin_y + ROW_SIZE + MARGIN)
        cv2.putText(annotated_image, result_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                    FONT_SIZE, TEXT_COLOR, FONT_THICKNESS)

    return annotated_image

# STEP 1: Set up FaceDetector
base_options = python.BaseOptions(model_asset_path='detector.tflite')
options = vision.FaceDetectorOptions(base_options=base_options)
detector = vision.FaceDetector.create_from_options(options)

# STEP 2: Open video file
cap = cv2.VideoCapture('stairs.mp4')

while cap.isOpened():
    success, frame_bgr = cap.read()
    if not success:
        break

    # Convert to RGB (MediaPipe expects RGB)
    frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)

    # Convert to MediaPipe Image
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

    # STEP 3: Detect faces
    detection_result = detector.detect(mp_image)

    # STEP 4: Visualize
    image_np = mp_image.numpy_view()
    annotated_image = visualize(image_np, detection_result)

    # Display the frame
    cv2.imshow("Detected Faces", annotated_image)
    if cv2.waitKey(33) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# REAL CODE

In [41]:
print("hllo")
import cv2 as cv
from threading import Thread
from queue import Queue
# from mtcnn.mtcnn import MTCNN
from facenet_pytorch import MTCNN
detector = MTCNN(keep_all=True, device=device)
import numpy as np
import pickle
from keras_facenet import FaceNet
from datetime import timedelta
import datetime
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
import torch
print(torch.cuda.is_available())
import os
import sys
import queue
print("OpenCL Available:", cv.ocl.haveOpenCL())
print("OpenCL in Use:", cv.ocl.useOpenCL())
sys.path.append(r"D:\Nexus AI\Nexus Face recognition\sort")
sort_path = os.path.join(os.getcwd(), 'sort')
sys.path.insert(0, sort_path)
from sort import Sort
import datetime
from threading import Event
import time
from config import RTSP_URL
from ultralytics import YOLO
from logging.handlers import RotatingFileHandler
import logging
from logging.handlers import RotatingFileHandler
import os
cap_face = []
# Queus for multi-threading
frame_queue = Queue(maxsize = 30) 
# result_queue = Queue(maxsize = 30)
face_crop_queue = Queue(maxsize = 20)
# face_crop_queue = queue.Queue()
stop_event = Event()
ymodel = YOLO("yolov8n.pt")
# Setup logs directory
if not os.path.exists('logs'):
    os.makedirs('logs')
# Setup logging
log_formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')
log_file = 'logs/attendance_logs.log'
file_handler = RotatingFileHandler(log_file, maxBytes=5*1024*1024, backupCount=3)  # 5MB per file, keep 3 backups
file_handler.setFormatter(log_formatter)
console_handler = logging.StreamHandler()
console_handler.setFormatter(log_formatter)
logging.basicConfig(level=logging.INFO, handlers=[file_handler, console_handler])
logger = logging.getLogger()
face_batch = []
# temp code
# DATA_DIR = './datatemp2'
# if not os.path.exists(DATA_DIR):
#     os.makedirs(DATA_DIR)

def get_batch_embeddings(face_batch):
    # Convert the list of face images to a NumPy array and ensure they are float32
    face_batch = np.array([face.astype("float32") for face in face_batch])
    embeddings = embedder.embeddings(face_batch)  # Predict embeddings for the batch
    return embeddings  # Return the embeddings for all faces in the batch
start = time.time()
# detector = MTCNN()
logging.info("MTCNN Loaded in %.2f seconds", time.time() - start)
#  loding models and encoders and facenet
start = time.time()
embedder = FaceNet()
logging.info("FaceNet Loaded in %.2f seconds", time.time() - start)
start = time.time()
with open('model/svm_model_160x160.pkl','rb') as f:
    model = pickle.load(f)
logging.info("SVM Loaded in %.2f seconds", time.time() - start)
start = time.time()
with open('model/label_encoder.pkl','rb') as g:
    encoder = pickle.load(g)
logging.info("encoder Loaded in %.2f seconds", time.time() - start)
# SORT Tracker
start = time.time()
tracker = Sort(max_age=5, min_hits=3, iou_threshold=0.2)
logging.info("tracker model Loaded in %.2f seconds", time.time() - start)
# Store recognized IDs and logs
track_id_to_name = {}
in_time_log = {}
out_time_log = {}
face_recognition_attempts = {}  # track_id -> count # track_id -> name
batch_id = []
already_recognized =[]
employees = {'parn':False, 'Adivya':False}
timeduration = {'parn':[], 'Adivya':[]}
newemployee = {'parn':[],'Adivya':[]}
BATCH_SIZE = 5
last_seen_frame = {}
frame_count = 0
inactive_threshold = 5
# disappear_threshold = 5
# for recorded vdo testing
# rec_vdo = cv.VideoCapture('output_video.avi')
# for inbuilt camera
# in_cam = cv.VideoCapture(0)
# for extrnal camera

class CameraStream:
    def __init__(self, rtsp_url):
        self.rtsp_url = rtsp_url
        self.cap = cv.VideoCapture(rtsp_url)
    def read(self):
        return self.cap.read()
    def reconnect(self):
        self.cap.release()
        time.sleep(3)
        self.cap = cv.VideoCapture(self.rtsp_url)
    def is_opened(self):
        return self.cap.isOpened()
    def release(self):
        self.cap.release()
    def sett(self, exposure=-6, fps=30, width=1280, height=720):
        self.cap.set(cv.CAP_PROP_EXPOSURE, exposure)  # Lower is faster (e.g., -6, -7)
        self.cap.set(cv.CAP_PROP_FPS, fps)
        self.cap.set(cv.CAP_PROP_FRAME_WIDTH, width)
        self.cap.set(cv.CAP_PROP_FRAME_HEIGHT, height)
new_width = 1280
new_height = 720
# ext_cam = cv.VideoCapture(RTSP_URL)
ext_cam = CameraStream(RTSP_URL)
ext_cam.sett(exposure=-6, fps=30)

# ext_cam.set(cv.CAP_PROP_EXPOSURE, -5)

def process_frame_gpu(frame):
    # Convert frame to UMat for GPU processing
    frame_umat = cv.UMat(frame)
    # Resize the frame on GPU
    frame_umat = cv.resize(frame_umat, (1280, 720))
    return frame_umat
def apply_clahe(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)
    return cv.cvtColor(enhanced, cv.COLOR_GRAY2BGR)

def capturing_frames(vdosource,new_width,new_height):
    frame_count = 0
    saved_count = 0
    # file_path = os.path.join(DATA_DIR, f'{saved_count}.jpg')
    logger.debug("Capturing frame from video stream.")
    while not stop_event.is_set():
        res,frame = vdosource.read()
        if not res:
            print("[WARNING] Camera disconnected. Attempting to reconnect...")
            time.sleep(1) 
            vdosource.reconnect()   
            continue        #Later I wall add limiting retry and add delay to previent infinite loop
        frame = cv.resize(frame,(new_width, new_height))
        # kernel = np.array([[0, -1, 0],
        #         [-1, 5,-1],
        #         [0, -1, 0]])
        # sharpened = cv.filter2D(frame, -1, kernel)
        # frame_umat = process_frame_gpu(sharpened)
        # frame = apply_clahe(frame)
        frame_umat = process_frame_gpu(frame)
        if not frame_queue.full():
            # file_path = os.path.join(DATA_DIR, f'{saved_count}.jpg')  # <-- Moved inside loop
            # cv.imwrite(file_path, frame_umat)
            # saved_count+=1
            frame_queue.put(frame_umat)

def detection_and_tracking():
    save_count = 0
    global frame_count, last_seen_frame
    while not stop_event.is_set():
        if not frame_queue.empty():
            frame_count += 1
            frame = frame_queue.get()
            frame_np = frame.get()  
            clean_frame = frame_np.copy()
            results = ymodel.predict(source=frame_np, verbose=False)[0]
            detections = []
            for r in results.boxes.data.tolist():
                x1, y1, x2, y2, score, cls = r
                if int(cls) == 0 and score > 0.5:
                    detections.append([x1, y1, x2, y2, score])
            tracked = []
            if detections:
                tracked = tracker.update(np.array(detections))
            current_ids = set()
            for d in tracked:
                x1, y1, x2, y2, track_id = [int(v) for v in d]
                current_ids.add(track_id)
                last_seen_frame[track_id] = frame_count
                person_crop = frame_np[y1:y2, x1:x2]
                if person_crop is None or person_crop.size == 0:
                    continue
                rgb_crop = cv.cvtColor(person_crop, cv.COLOR_BGR2RGB)
                faces = detector.detect_faces(rgb_crop)
                if len(faces) > 0:
                    face = faces[0]
                    fx, fy, fw, fh = face['box']
                    fx, fy = abs(fx), abs(fy)
                    face_region = rgb_crop[fy:fy+fh, fx:fx+fw]
                    face_region = cv.resize(face_region, (160, 160))
                    if track_id not in already_recognized:
                        cap_face.append(face_region)
                        face_crop_queue.put((face_region, track_id))

                name = track_id_to_name.get(track_id, "Unknown")
                # label = f"{name} | ID: {track_id}"
                # cv.rectangle(frame_np, (x1, y1), (x2, y2), (0, 255, 0), 2)
                # cv.putText(frame_np, label, (x1, y1 - 10),
                        #   cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
            # Cleanup inactive IDs
            for rid in list(already_recognized):
                if rid not in current_ids and frame_count - last_seen_frame.get(rid, 0) > inactive_threshold:
                    already_recognized.remove(rid)
                    track_id_to_name.pop(rid, None)
                    logger.info(f"Removed ID {rid} due to inactivity.")

            cv.imshow('Face Recognition + Tracking', frame_np)
            if cv.waitKey(1) & 0xFF == ord('q'):
                stop_event.set()
                break

def face_recognition_and_logging():
    face_batch = []
    batch_id = []

    while not stop_event.is_set():
        try:
            face_region, track_id = face_crop_queue.get(timeout=1)
            face_batch.append(face_region)
            batch_id.append(track_id)

            if len(face_batch) >= BATCH_SIZE:
                embeddings = get_batch_embeddings(face_batch)

                for i, emb in enumerate(embeddings):
                    proba = model.predict_proba([emb])[0]
                    best_idx = np.argmax(proba)
                    confidence = proba[best_idx]

                    if confidence < 0.93:
                        predicted_name = "Unknown"
                    else:
                        predicted_name = encoder.inverse_transform([best_idx])[0]

                    now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

                    if predicted_name != "Unknown":
                        logger.info(f"{predicted_name} recognized with ID {batch_id[i]}")
                        newemployee.setdefault(predicted_name, []).append(now)
                        logger.info(f"logged for {predicted_name} at {now}")
                        track_id_to_name[batch_id[i]] = predicted_name
                        already_recognized.append(batch_id[i])

                        if batch_id[i] not in in_time_log:
                            in_time_log[batch_id[i]] = now
                        out_time_log[batch_id[i]] = now

                face_batch = []
                batch_id = []

        except queue.Empty:
            continue

def final_output():
    def calculate_total_duration(timestamps):
        total_seconds = 0
        for i in range(0, len(timestamps) - 1, 2):  # step by 2
            in_time = datetime.datetime.strptime(timestamps[i], "%Y-%m-%d %H:%M:%S")
            out_time = datetime.datetime.strptime(timestamps[i+1], "%Y-%m-%d %H:%M:%S")
            total_seconds += int((out_time - in_time).total_seconds())
        minutes = total_seconds // 60
        seconds = total_seconds % 60
        return minutes, seconds
    logger.info("Final attendance summary:")
    for employee, timestamps in newemployee.items():
        mins, secs = calculate_total_duration(timestamps)
        logger.info(f"{employee} stayed for {mins} min {secs} sec")
if __name__ == "__main__":
    video_source = ext_cam
# ext_cam, rec_vdo, in_cam
    capture_thread = Thread(target=capturing_frames, args=(video_source, new_width, new_height), daemon=True)
    logger.info("capture_thread started")
    detection_thread = Thread(target=detection_and_tracking)
    recognition_thread = Thread(target=face_recognition_and_logging)
    capture_thread.start()
    detection_thread.start()
    recognition_thread.start()
    try:
        while not stop_event.is_set():
            if cv.waitKey(1) & 0xFF == ord('q'):
                stop_event.set()
                break
    except KeyboardInterrupt:
        logger.info("Interrupted by user")
        stop_event.set()
    capture_thread.join()
    detection_thread.join()
    recognition_thread.join()
    video_source.release()
    cv.destroyAllWindows()
    print("i am hereeee")
    final_output()

hllo
[]
False
OpenCL Available: True
OpenCL in Use: True


2025-05-02 12:27:43,177 | INFO | MTCNN Loaded in 0.27 seconds
2025-05-02 12:27:43,178 | INFO | Loading weights.
2025-05-02 12:27:43,180 | INFO | Looking for C:\Users\Parn/.keras-facenet\20180402-114759\20180402-114759-weights.h5
2025-05-02 12:27:46,113 | INFO | FaceNet Loaded in 2.93 seconds
C:\Python compiler\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator SVC from version 1.6.1 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
2025-05-02 12:27:46,116 | INFO | SVM Loaded in 0.00 seconds
C:\Python compiler\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-lear

1/1 [==============================] - 4s 4s/step


2025-05-02 12:28:01,634 | INFO | parn recognized with ID 168
2025-05-02 12:28:01,638 | INFO | logged for parn at 2025-05-02 12:28:01
2025-05-02 12:28:01,643 | INFO | parn recognized with ID 168
2025-05-02 12:28:01,646 | INFO | logged for parn at 2025-05-02 12:28:01


1/1 [==============================] - 0s 247ms/step


...

1/1 [==============================] - 0s 267ms/step


2025-05-02 12:28:11,890 | INFO | parn recognized with ID 168
2025-05-02 12:28:11,892 | INFO | logged for parn at 2025-05-02 12:28:11
2025-05-02 12:28:11,896 | INFO | parn recognized with ID 168
2025-05-02 12:28:11,897 | INFO | logged for parn at 2025-05-02 12:28:11
2025-05-02 12:28:11,900 | INFO | parn recognized with ID 168
2025-05-02 12:28:11,902 | INFO | logged for parn at 2025-05-02 12:28:11
2025-05-02 12:28:11,907 | INFO | parn recognized with ID 168
2025-05-02 12:28:11,909 | INFO | logged for parn at 2025-05-02 12:28:11
2025-05-02 12:28:11,917 | INFO | parn recognized with ID 168
2025-05-02 12:28:11,921 | INFO | logged for parn at 2025-05-02 12:28:11
2025-05-02 12:28:19,819 | INFO | Removed ID 168 due to inactivity.
2025-05-02 12:28:19,822 | INFO | Removed ID 168 due to inactivity.
2025-05-02 12:28:19,824 | INFO | Removed ID 168 due to inactivity.
2025-05-02 12:28:19,826 | INFO | Removed ID 168 due to inactivity.
2025-05-02 12:28:19,827 | INFO | Removed ID 168 due to inactivity.


1/1 [==============================] - 0s 272ms/step


2025-05-02 12:28:25,751 | INFO | parn recognized with ID 169
2025-05-02 12:28:25,753 | INFO | logged for parn at 2025-05-02 12:28:25
2025-05-02 12:28:25,758 | INFO | parn recognized with ID 169
2025-05-02 12:28:25,763 | INFO | logged for parn at 2025-05-02 12:28:25
2025-05-02 12:28:25,767 | INFO | parn recognized with ID 169
2025-05-02 12:28:25,769 | INFO | logged for parn at 2025-05-02 12:28:25
2025-05-02 12:28:25,773 | INFO | parn recognized with ID 169
2025-05-02 12:28:25,776 | INFO | logged for parn at 2025-05-02 12:28:25
2025-05-02 12:28:25,781 | INFO | parn recognized with ID 169
2025-05-02 12:28:25,784 | INFO | logged for parn at 2025-05-02 12:28:25
2025-05-02 12:28:26,818 | INFO | Final attendance summary:
2025-05-02 12:28:26,821 | INFO | parn stayed for 0 min 9 sec
2025-05-02 12:28:26,823 | INFO | Adivya stayed for 0 min 0 sec


i am hereeee


In [39]:
print("Hwllo")

Hwllo


In [42]:
cap_face

[array([[[ 33,  36,  39],
         [ 33,  36,  39],
         [ 32,  35,  38],
         ...,
         [ 41,  38,  37],
         [ 39,  37,  36],
         [ 39,  37,  36]],
 
        [[ 33,  36,  39],
         [ 33,  36,  39],
         [ 32,  35,  38],
         ...,
         [ 41,  39,  38],
         [ 39,  37,  36],
         [ 39,  37,  36]],
 
        [[ 34,  37,  40],
         [ 34,  37,  40],
         [ 33,  36,  39],
         ...,
         [ 43,  41,  39],
         [ 40,  38,  36],
         [ 39,  37,  35]],
 
        ...,
 
        [[138, 147, 172],
         [138, 147, 172],
         [138, 147, 172],
         ...,
         [ 91,  83,  86],
         [ 95,  88,  91],
         [ 95,  89,  92]],
 
        [[138, 147, 172],
         [138, 147, 172],
         [138, 147, 172],
         ...,
         [ 94,  87,  90],
         [ 98,  93,  95],
         [ 98,  94,  96]],
 
        [[138, 147, 172],
         [138, 147, 172],
         [138, 147, 172],
         ...,
         [ 95,  88,  91],
  

In [46]:
import cv2 as cv

# Assuming 'cropped_faces' is a list of images (NumPy arrays)
# For testing purposes, you can replace it with your list of cropped faces.
# cropped_faces = []  # Replace this with your list of cropped face images

# Check if the list is empty
if not cap_face:
    print("No cropped faces available to view.")
else:
    index = 0  # Start from the first image in the list
    while True:
        # Display the current cropped face
        cv.imshow(f'Cropped Face {index+1}', cap_face[index])
        print(f"Viewing Cropped Face {index+1}")

        # Wait for a key press
        key = cv.waitKey(0) & 0xFF

        if key == ord('n'):  # Press 'n' to view the next image
            index += 1
            if index >= len(cap_face):  # If we reach the end of the list
                print("No more faces to view.")
                break
        elif key == ord('p'):  # Press 'p' to view the previous image
            index -= 1
            if index < 0:  # Prevent going out of bounds
                print("This is the first face.")
                index = 0
        elif key == ord('q'):  # Press 'q' to quit viewing
            print("Exiting view mode.")
            break

        # Close the window after key press
        cv.destroyAllWindows()

# Release any remaining OpenCV windows
cv.destroyAllWindows()


Viewing Cropped Face 1
Viewing Cropped Face 2
Viewing Cropped Face 3
Viewing Cropped Face 2
Viewing Cropped Face 3
Viewing Cropped Face 4
Viewing Cropped Face 5
Viewing Cropped Face 6
Viewing Cropped Face 7
Viewing Cropped Face 8
Viewing Cropped Face 9
Viewing Cropped Face 10
Viewing Cropped Face 11
Viewing Cropped Face 12
Viewing Cropped Face 13
Viewing Cropped Face 14
Viewing Cropped Face 15
Viewing Cropped Face 16
Viewing Cropped Face 17
Viewing Cropped Face 18
Viewing Cropped Face 19
Viewing Cropped Face 20
No more faces to view.


In [ ]:
n